# Big Data w biznesie

## Czwarty notebook

In [ ]:
from IPython.core.display import HTML


def _set_css_style(css_file_path):
    """
    Read the custom CSS file and load it into Jupyter.
    Pass the file path to the CSS file.
    """

    styles = open(css_file_path, "r").read()
    s = '<style>%s</style>' % styles
    return HTML(s)


_set_css_style("../custom.css")

Pierwszym krokiem, jak zawsze, jest zaimportowanie potrzebnych nam bibliotek.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
df = pd.read_csv("./cars.csv")

Tym razem nauczymy się dodawać widgety i łączyć je z innymi obiektami.

Sugeruję wykonywanie notebooków z tego tygodnia na stworzonym serwerze Jupytera lub w Google Colabie. W PyCharmie (nie wiem, jak jest w VSCodzie), wiele z widgetów może nie działać poprawnie i mogą pojawić się problemy z interakcjami.

Wykorzystamy do tego bibliotekę `ipywidgets`, którą najpierw zaimportujemy.

In [ ]:
import ipywidgets as widgets

Jest wiele widgetów, z których możemy skorzystać. Pełną listę można znaleźć w dokumentacji bądź przy pomocy następującego polecenia:

In [ ]:
print(dir(widgets))

Zaczniemy od dodania pierwszego widgetu - suwaka.

In [ ]:
widgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='Suwak:',
    value=5
)

Jeśli chcemy poznać dostępne właściwości i metody klasy `widgets.IntSlider` (lub innej klasy) możemy to zrobić w następujący sposób. Funkcja `dir()` wyświetla listę nazw atrybutów i metod dostępnych dla tej klasy. Warto z tego skorzystać, jeśli chcemy dowiedzieć się, jak można wykorzystać daną klasę.

In [ ]:
dir(widgets.IntSlider)

Jeśli na tej liście znajdziemy metodę, która nas interesuje, możemy mieć wgląd w jej dokumentacje w następujący sposób:

In [ ]:
print(widgets.IntSlider.observe.__doc__)

Wracając do wykorzystywania widgetów, aby nasz obiekt był odpowiednio wyświetlany wykorzystamy do tego bibliotekę `IPython`.

Importując `display` z modułu `IPython`, możemy wyświetlić wyniki i obiekty w interaktywny sposób w notebooku Jupyter.

In [ ]:
from IPython.display import display

In [ ]:
slider = widgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='Suwak:',
    value=5
)
display(slider)

W ten sposób możemy wyświetlić dowolny obiekt za pomocą metody `display`.

Istnieje też możliwość stylowania widgetów. Na przykład, spróbujmy stworzyć nowy widget z nieco dłuższym opisem.

In [ ]:
slider_long = widgets.IntSlider(
    description='To jest suwak:'
)
display(slider_long)

W takim przypadku opis zostaje skrócony. Chcemy zobaczyć pełny opis, dlatego możemy skorzystać z poniższego kodu:

In [ ]:
style_long = {'description_width': 'initial'}
slider_long = widgets.IntSlider(
    description='To jest suwak:',
    style=style_long
)
display(slider_long)

Jeśli chcemy poznać wszystkie dostępne opcje w słowniku `style` dla danego obiektu, możemy użyć następującej komendy:

In [ ]:
slider_long.style.keys

Inną ważną właściwość, którą posiada nasz obiekt (suwak) jest `value`. Możemy się do niej łatwo odwołać.

In [ ]:
slider.value

Co więcej, możemy ją też łatwo zmienić.

In [ ]:
slider.value = 1

Istnieje możliwość zsynchronizować ze sobą dwóch widgetów.

In [ ]:
slider = widgets.IntSlider()
text = widgets.IntText()
display(slider, text)
widgets.jslink((slider, 'value'), (text, 'value'))

Metoda `jslink` jest używana do zsynchronizowania wartości `value` między nimi, co oznacza, że jeśli zmienisz wartość jednego z widgetów, wartość drugiego widgetu również się zmieni, aby odzwierciedlić tę samą wartość.

Drugim widgetem, który stworzymy jest przycisk (Button).

In [ ]:
btn = widgets.Button(description='Normalny')
display(btn)

Widgety pozwalają na interakcję z użytkownikiem, np. reagowanie na kliknięcia czy zmiany wartości. W tym przykładzie tworzymy przycisk i definiujemy funkcję `btn_eventhandler`, która będzie wywoływana po kliknięciu przycisku i wyświetlała odpowiedni komunikat. Następnie dodajemy funkcję do przycisku używając metody `on_click`.

In [ ]:
def btn_eventhandler(obj):
    print(f'Cześć, tutaj Twój {obj.description} guzik!')

btn.on_click(btn_eventhandler)
display(btn)

Kolejnym widgetem, którego użyjemy, jest rozwijana lista (Dropdown):

In [ ]:
widgets.Dropdown(options=['nic', 'coś', 'wszystko'])

Będziemy chcieli połączyć `Dropdown` z DataFrame'em zawierającym dane z pliku `cars.csv`. Możemy stworzyć rozwijaną listę, która będzie filtrowała nasze dane po roku produkcji.

Aby to zrobić, najpierw potrzebna jest nam funkcja, która zwraca listę zawierającą unikalne wartości kolumny oraz opcję 'All':

In [ ]:
def unique_sorted_values_plus_all(array):
    unique = array.unique().tolist()
    unique.sort(reverse=True)
    unique.insert(0, 'All')
    return unique

Tak wygląda tworzenie w ten sposób rozwijanej listy:

In [ ]:
dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_all(df['year_produced']))
display(dropdown_year)

 Potrzebujemy funkcji, która reaguje na zmiany i będzie filtrowała dane w zależności od wartości.

In [ ]:
def dropdown_year_eventhandler(change):
    if change.new == 'All':
        display(df)
    else:
        display(df[df['year_produced'] == change.new])

Zwróć uwagę, że argumentem tej funkcji jest `change`, który zawiera informacje o zmianach, dzięki czemu możemy uzyskać dostęp do nowej wartości (`change.new`).

Poza tym `Dropdown`, tak jak większość widgetów, udostępnia metodę `observe`. Jako argument przyjmuje funkcję, która zostanie wywołana, gdy wartość rozwijanej listy zostanie zmieniona.

In [ ]:
dropdown_year.observe(dropdown_year_eventhandler, names='value')

Ostatecznie wyświetlamy naszą rozwijaną listę za pomocą polecenia:

In [ ]:
display(dropdown_year)

Niestety, wyniki wszystkich zapytań gromadzą i pojawiają się jeden pod drugim.

Chcielibyśmy odświeżać zawartość komórki za każdym razem i wyświetlać nowy DataFrame.

W rozwiązaniu tego problemu pomoże nam kolejny widget, czyli Output.

In [ ]:
output_year = widgets.Output()

Pozwala na tworzenie kontenera, w którym można wyświetlać wyjście z innych widgetów lub kodu. Można go wykorzystać do wyświetlenia wyników zapytań lub do tworzenia interaktywnych interfejsów użytkownika. Kontener ten umożliwia też dynamiczne usuwanie lub czyszczenie zawartości i zastępowanie go nową treścią.

Użyjemy go w następujący sposób.

Po pierwsze przy każdym wywołaniu będziemy czyścić wyjście za pomocą funkcji `clear_output`.

Po drugie użyjemy składni `with`, która pozwala obsłużyć otwarcie i zamknięcie pliku lub innych zasobów, które wymagają tego typu zachowania.
Użycie `with output_year` oznacza, że wszystko, co zostanie wyświetlone wewnątrz bloku `with`, zostanie przekierowane do tego widgetu. Dzięki temu można z łatwością zmieniać wyświetlane dane bez konieczności ręcznego czyszczenia widgetu i bezpośredniego wyświetlania danych w danym bloku kodu.

In [ ]:
def dropdown_year_eventhandler(change):
    output_year.clear_output()
    with output_year:
        if change.new == 'All':
            display(df)
        else:
            display(df[df['year_produced'] == change.new])

Całość wygląda następująco:

In [ ]:
dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_all(df['year_produced']))

output_year = widgets.Output()

def dropdown_year_eventhandler(change):
    output_year.clear_output()
    with output_year:
        if change.new == 'All':
            display(df)
        else:
            display(df[df['year_produced'] == change.new])

dropdown_year.observe(dropdown_year_eventhandler, names='value')
display(dropdown_year)
display(output_year)

W następnym kroku połączymy ze sobą wyjścia (Outputy) z dwóch różnych widgetów.

Oprócz filtrowania po roku produkcji chcemy również móc filtrować po kolorze samochodu.

Stworzymy więc nową rozwijaną listę, która będzie zawierać wszystkie dostępne kolory.

In [ ]:
dropdown_color = widgets.Dropdown(options = unique_sorted_values_plus_all(df['color']))

Musimy też dodać funkcję, która filtruje nasze dane na podstawie dwóch parametrów.

In [ ]:
def common_filtering(year, color):
    output.clear_output()

    if year == 'All': year = df['year_produced'].unique().tolist()
    if color == 'All': color = df['color'].unique().tolist()

    common_filter = df.query('year_produced == @year and color == @color')

    with output:
        display(common_filter)

Zadaniem obu funkcji reagujących na zmiany jest przefiltrowanie danych po pierwszym parametrze, który zmieniamy i przekazanie drugiego parametru, który pozostaje taki sam.

In [ ]:
def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_color.value)

def dropdown_color_eventhandler(change):
    common_filtering(dropdown_year.value, change.new)

Musimy też dodać funkcję, która obserwuje zmiany.

In [ ]:
dropdown_year.observe(
dropdown_year_eventhandler, names='value')

dropdown_color.observe(
dropdown_color_eventhandler, names='value')

Całość wygląda następująco:

In [ ]:
dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_all(df['year_produced']))

dropdown_color = widgets.Dropdown(options = unique_sorted_values_plus_all(df['color']))

output = widgets.Output()

def common_filtering(year, color):
    output.clear_output()

    if year == 'All': year = df['year_produced'].unique().tolist()
    if color == 'All': color = df['color'].unique().tolist()

    common_filter = df.query('year_produced == @year and color == @color')

    with output:
        display(common_filter)

def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_color.value)

def dropdown_color_eventhandler(change):
    common_filtering(dropdown_year.value, change.new)

dropdown_year.observe(dropdown_year_eventhandler, names='value')

dropdown_color.observe(dropdown_color_eventhandler, names='value')

display(dropdown_year)
display(dropdown_color)
display(output)

Na ten moment jesteśmy w stanie wyświetlać i filtrować dane po parametrach. Kolejnym krokiem będzie kolorowanie danych po wartościach numerycznych.

W naszym przypadku będziemy chcieli kolorować wartości w kolumnie z ceną samochodu w zależności od tego, czy będą większe lub mniejsze od wybranej przez nas wartości.

Do tego przyda nam się widget `BoundedFloatText`, za pomocą którego będziemy sterować wybraną przez nas wartością.

In [ ]:
bounded_num = widgets.BoundedFloatText(min=0, max=60000, value=10000, step=1000)
display(bounded_num)

Potrzebujemy też funkcji, która będzie porównywać wartości i zwracać kolor czerwony albo czarny.

In [ ]:
def colour_ge_value(value, comparison):
    if value >= comparison:
        return 'color: red'
    else:
        return 'color: black'

W naszym przypadku będziemy stosować stylowanie na danych w odpowiedniej kolumnie za pomocą funkcji `style` i `applymap`.

In [ ]:
def common_filtering(year, color, num):
    output.clear_output()

    if year == 'All': year = df['year_produced'].unique().tolist()
    if color == 'All': color = df['color'].unique().tolist()

    common_filter = df.query('year_produced == @year and color == @color')

    with output:
        display(common_filter
                .style.applymap(
                    lambda x: colour_ge_value(x, num),
                    subset=['price_usd']))

Do naszych funkcji reagujących na zmiany, musimy dodać trzeci przekazywany parametr.

In [ ]:
def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_color.value, bounded_num.value)

def dropdown_color_eventhandler(change):
    common_filtering(dropdown_year.value, change.new, bounded_num.value)

Oraz musimy dodać trzecią funkcję reagującą na zmiany trzeciego parametru.

In [ ]:
def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_color.value, change.new)

Sprawdzanie, czy dane są większe lub mniejsze od 38 tys. wartości (tyle rekord znajduje się w pliku `cars.csv`) trwa bardzo długo i często powoduje, że program się zawiesza lub zacina.

Aby umożliwić bezproblemowe przetwarzanie danych, ograniczymy nasze dane do 5% oryginalnego rozmiaru.

In [ ]:
df_subset = df.sample(n=int(0.05*len(df))).copy()

Całość wygląda następująco:

In [ ]:
dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_all(df_subset['year_produced']))

dropdown_color = widgets.Dropdown(options = unique_sorted_values_plus_all(df_subset['color']))

bounded_num = widgets.BoundedFloatText(min=0, max=60000, value=10000, step=1000)

output = widgets.Output()

def common_filtering(year, color, num):
    output.clear_output()

    if year == 'All': year = df_subset['year_produced'].unique().tolist()
    if color == 'All': color = df_subset['color'].unique().tolist()

    common_filter = df_subset.query('year_produced == @year and color == @color')

    with output:
        display(common_filter
                .style.applymap(
                    lambda x: colour_ge_value(x, num),
                    subset=['price_usd']))

def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_color.value, bounded_num.value)

def dropdown_color_eventhandler(change):
    common_filtering(dropdown_year.value, change.new, bounded_num.value)

def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_color.value, change.new)

dropdown_year.observe(dropdown_year_eventhandler, names='value')

dropdown_color.observe(dropdown_color_eventhandler, names='value')

bounded_num.observe(bounded_num_eventhandler, names='value')

display(dropdown_year)
display(dropdown_color)
display(bounded_num)
display(output)

Kolejna funkcjonalność, którą dodamy to rysowanie wykresów.

Będziemy wyświetlać tzw. wykres gęstości jądrowej dla wartości przebiegu samochodów.

Funkcja `sns.kdeplot` służy do wyświetlania wykresów gęstości jądrowej (kernel density estimation plot). Ten typ wykresu przedstawia rozkład prawdopodobieństwa danych w sposób zbliżony do histogramu, ale bez dyskretyzacji. Zamiast tego, funkcja używa jądra (kernel) do oszacowania gęstości w każdym punkcie i przedstawia to na wykresie jako gładką krzywą.

In [ ]:
sns.kdeplot(df_subset['odometer_value'], fill=True)
plt.show()

Do wyświetlania wykresów stworzymy nowy `Output`.

In [ ]:
plot_output = widgets.Output()

Na końcu naszej funkcji filtrującej dodamy wyświetlanie tego wykresu.

In [ ]:
def common_filtering(year, color, num):
    output.clear_output()
    plot_output.clear_output()

    if year == 'All': year = df_subset['year_produced'].unique().tolist()
    if color == 'All': color = df_subset['color'].unique().tolist()

    common_filter = df_subset.query('year_produced == @year and color == @color')

    with output:
        display(common_filter
                .style.applymap(
                    lambda x: colour_ge_value(x, num),
                    subset=['price_usd']))

    with plot_output:
        sns.kdeplot(common_filter['odometer_value'], fill=True)
        plt.show()

Całość wygląda następująco:

In [ ]:
dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_all(df_subset['year_produced']))

dropdown_color = widgets.Dropdown(options = unique_sorted_values_plus_all(df_subset['color']))

bounded_num = widgets.BoundedFloatText(min=0, max=60000, value=10000, step=1000)

output = widgets.Output()
plot_output = widgets.Output()


def common_filtering(year, color, num):
    output.clear_output()
    plot_output.clear_output()

    if year == 'All': year = df_subset['year_produced'].unique().tolist()
    if color == 'All': color = df_subset['color'].unique().tolist()

    common_filter = df_subset.query('year_produced == @year and color == @color')

    with output:
        display(common_filter
                .style.applymap(
                    lambda x: colour_ge_value(x, num),
                    subset=['price_usd']))

    with plot_output:
        sns.kdeplot(common_filter['odometer_value'], fill=True)
        plt.show()

def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_color.value, bounded_num.value)

def dropdown_color_eventhandler(change):
    common_filtering(dropdown_year.value, change.new, bounded_num.value)

def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_color.value, change.new)

dropdown_year.observe(dropdown_year_eventhandler, names='value')

dropdown_color.observe(dropdown_color_eventhandler, names='value')

bounded_num.observe(bounded_num_eventhandler, names='value')

display(dropdown_year)
display(dropdown_color)
display(bounded_num)
display(output)
display(plot_output)

Na koniec połączymy wszystko w jeden `Dashboard`, czyli interaktywny interfejs użytkownika, który umożliwia wyświetlanie i manipulowanie danymi w sposób graficzny i intuicyjny.

Nasze dane wejściowe (Inputy) połączymy w jeden widget o nazwie `HBox`, który ustawia je poziomo.

In [ ]:
input_widgets = widgets.HBox(
[dropdown_year, dropdown_color, bounded_num])
display(input_widgets)

A nasze dane wyjściowe (Outputy) połączymy w jeden widget o nazwie `Tab`, w którym każde wyjście będzie wyświetlane w innej karcie.

In [ ]:
tab = widgets.Tab([output, plot_output])
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'KDE Plot')
display(tab)

Całość połączymy w jeden Dashboard.

In [ ]:
dashboard = widgets.VBox([input_widgets, tab])
display(dashboard)

Ostatnim krokiem będzie poprawienie naszego Dashboardu poprzez dodanie trochę przestrzeni. Ostatni widget w tym notebooku - `Layout`, który doda margines 50 pikseli między elementami.

In [ ]:
item_layout = widgets.Layout(margin='0 0 50px 0')

Więc po raz ostatni, całość wygląda następująco:

In [ ]:
input_widgets = widgets.HBox([dropdown_year, dropdown_color, bounded_num], layout=item_layout)
tab = widgets.Tab([output, plot_output], layout=item_layout)
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'KDE Plot')
dashboard = widgets.VBox([input_widgets, tab])
display(dashboard)